cant find a better way to compute the reduced density matrix.

In [2]:
using Test, LinearAlgebra

function compute_concurrence(ρ)
    """Helper function to compute concurrence for testing"""
    σ_y = [0 -im; im 0]
    spin_flip = kron(σ_y, σ_y)
    ρ_tilde = spin_flip * conj(ρ) * spin_flip
    R = ρ * ρ_tilde
    λ = eigvals(R)
    λ_real = real(λ)
    λ_clamped = max.(λ_real, 0.0)
    λ_sorted = sort(λ_clamped, rev=true)
    C = max(0, sqrt(λ_sorted[1]) - sqrt(λ_sorted[2]) - sqrt(λ_sorted[3]) - sqrt(λ_sorted[4]))
    return C
end

function test_rdm(func)
    @testset "Comprehensive RDM Tests" begin
        
        # Test 1: Product state |↑↓⟩
        @testset "Product State" begin
            sites = siteinds("S=1/2", 2)
            ψ = MPS(sites, ["Up", "Dn"])
            ρ = func(ψ, sites, 1, 2)
            
            @test isapprox(tr(ρ), 1.0, atol=1e-10)
            @test isapprox(ρ, ρ', atol=1e-10)
            
            # Should be pure state
            eigenvals = real(eigvals(ρ))
            sort!(eigenvals, rev=true)
            @test isapprox(eigenvals[1], 1.0, atol=1e-10)
            @test all(abs.(eigenvals[2:end]) .< 1e-10)
            
            # No entanglement
            C = compute_concurrence(ρ)
            @test isapprox(C, 0.0, atol=1e-10)
        end
        
        # Test 2: Bell state - maximal entanglement
        @testset "Bell State (Maximal Entanglement)" begin
            sites = siteinds("S=1/2", 2)
            up_dn = MPS(sites, ["Up", "Dn"])
            dn_up = MPS(sites, ["Dn", "Up"])
            normalize!(up_dn)
            normalize!(dn_up)
            bell = add(up_dn, dn_up)
            normalize!(bell)
            
            ρ = func(bell, sites, 1, 2)
            
            @test isapprox(tr(ρ), 1.0, atol=1e-10)
            @test isapprox(ρ, ρ', atol=1e-10)
            
            # Check off-diagonal elements (signature of entanglement)
            @test sum(abs.(ρ - Diagonal(diag(ρ)))) > 1e-10
            
            # Maximal concurrence
            C = compute_concurrence(ρ)
            @test isapprox(C, 1.0, atol=1e-10)
        end
        
        # Test 3: GHZ state - no bipartite entanglement!
        @testset "GHZ State (Tripartite Only)" begin
            sites = siteinds("S=1/2", 3)
            uuu = MPS(sites, ["Up", "Up", "Up"])
            ddd = MPS(sites, ["Dn", "Dn", "Dn"])
            normalize!(uuu)
            normalize!(ddd)
            ghz = add(uuu, ddd)
            normalize!(ghz)
            
            # Test sites 1 and 3 (non-adjacent)
            ρ_13 = func(ghz, sites, 1, 3)
            
            @test isapprox(tr(ρ_13), 1.0, atol=1e-10)
            @test isapprox(ρ_13, ρ_13', atol=1e-10)
            
            # IMPORTANT: GHZ states show NO bipartite entanglement!
            # Any 2-qubit reduced density matrix is maximally mixed: ρ = I/2
            C_13 = compute_concurrence(ρ_13)
            @test isapprox(C_13, 0.0, atol=1e-10)
            
            # Test sites 1 and 2 (adjacent) - also no bipartite entanglement
            ρ_12 = func(ghz, sites, 1, 2)
            C_12 = compute_concurrence(ρ_12)
            @test isapprox(C_12, 0.0, atol=1e-10)
            
            # Symmetry
            ρ_21 = func(ghz, sites, 2, 1)
            @test isapprox(ρ_12, ρ_21, atol=1e-10)
        end
        
        # Test 4: Bell pairs - non-adjacent entanglement
        @testset "Two Bell Pairs (Non-Adjacent)" begin
            sites = siteinds("S=1/2", 4)
            # |Ψ⟩ = (|↑↓⟩ + |↓↑⟩)₁₂ ⊗ (|↑↓⟩ + |↓↑⟩)₃₄
            # Sites 1-2 entangled, 3-4 entangled, but 1-3 not entangled
            udud = MPS(sites, ["Up", "Dn", "Up", "Dn"])
            dudu = MPS(sites, ["Dn", "Up", "Dn", "Up"])
            uddu = MPS(sites, ["Up", "Dn", "Dn", "Up"])
            duud = MPS(sites, ["Dn", "Up", "Up", "Dn"])
            
            normalize!(udud)
            normalize!(dudu)
            normalize!(uddu)
            normalize!(duud)
            
            # Equal superposition of all four
            bell_pairs = add(add(add(udud, dudu), uddu), duud)
            normalize!(bell_pairs)
            
            # Sites 1-2 should be entangled
            ρ_12 = func(bell_pairs, sites, 1, 2)
            C_12 = compute_concurrence(ρ_12)
            @test C_12 > 0.8
            
            # Sites 1-3 should NOT be entangled (different Bell pairs)
            ρ_13 = func(bell_pairs, sites, 1, 3)
            C_13 = compute_concurrence(ρ_13)
            @test C_13 < 0.2
        end
        
        # Test 5: Antiferromagnetic chain - realistic system
        @testset "Antiferromagnetic Chain" begin
            N = 6
            sites = siteinds("S=1/2", N; conserve_qns=true)
            init_state = [isodd(i) ? "Up" : "Dn" for i in 1:N]
            ψ = MPS(sites, init_state)
            
            # Create simple Heisenberg Hamiltonian
            ampo = OpSum()
            for i = 1:(N-1)
                ampo += 0.5, "S+", i, "S-", i+1
                ampo += 0.5, "S-", i, "S+", i+1
                ampo += "Sz", i, "Sz", i+1
            end
            H = MPO(ampo, sites)
            
            # Find ground state
            sweeps = Sweeps(5)
            setmaxdim!(sweeps, 10, 20, 50)
            setcutoff!(sweeps, 1E-8)
            _, ψ_gs = dmrg(H, ψ, sweeps; outputlevel=0)
            
            # Test adjacent sites
            ρ_12 = func(ψ_gs, sites, 1, 2)
            @test isapprox(tr(ρ_12), 1.0, atol=1e-8)
            @test isapprox(ρ_12, ρ_12', atol=1e-8)
            
            # Adjacent sites should be more entangled than distant sites
            C_12 = compute_concurrence(ρ_12)
            ρ_14 = func(ψ_gs, sites, 1, 4)
            C_14 = compute_concurrence(ρ_14)
            @test C_12 > C_14  # Nearest neighbors more entangled
        end
        
        # Test 6: W state - different entanglement structure
        @testset "W State" begin
            sites = siteinds("S=1/2", 3)
            udd = MPS(sites, ["Up", "Dn", "Dn"])
            dud = MPS(sites, ["Dn", "Up", "Dn"])
            ddu = MPS(sites, ["Dn", "Dn", "Up"])
            normalize!(udd)
            normalize!(dud)
            normalize!(ddu)
            w_state = add(add(udd, dud), ddu)
            normalize!(w_state)
            
            ρ_12 = func(w_state, sites, 1, 2)
            C_12 = compute_concurrence(ρ_12)
            
            # W state has some bipartite entanglement (unlike GHZ)
            @test 0.4 < C_12 < 0.8
            @test isapprox(tr(ρ_12), 1.0, atol=1e-10)
        end
        
        # Test 7: Mixed state (after partial trace)
        @testset "Partial Trace Produces Mixed State" begin
            sites = siteinds("S=1/2", 4)
            # Create entangled state across all 4 sites
            uuuu = MPS(sites, ["Up", "Up", "Up", "Up"])
            dddd = MPS(sites, ["Dn", "Dn", "Dn", "Dn"])
            normalize!(uuuu)
            normalize!(dddd)
            psi_4 = add(uuuu, dddd)
            normalize!(psi_4)
            
            # Trace out sites 3 and 4, look at 1 and 2
            ρ_12 = func(psi_4, sites, 1, 2)
            
            # This should NOT be a pure state (mixed after partial trace)
            eigenvals = real(eigvals(ρ_12))
            sort!(eigenvals, rev=true)
            num_significant = sum(eigenvals .> 1e-10)
            @test num_significant > 1  # Mixed state has multiple eigenvalues
        end
        
        # Test 8: Consistency check - different site orderings
        @testset "Site Order Consistency" begin
            sites = siteinds("S=1/2", 4)
            psi = randomMPS(sites, 2)
            
            ρ_12 = func(psi, sites, 1, 2)
            ρ_21 = func(psi, sites, 2, 1)
            ρ_23 = func(psi, sites, 2, 3)
            ρ_32 = func(psi, sites, 3, 2)
            
            @test isapprox(ρ_12, ρ_21, atol=1e-10)
            @test isapprox(ρ_23, ρ_32, atol=1e-10)
        end
        
    end
end

test_rdm (generic function with 1 method)

In [3]:
using ITensors, ITensorMPS
using LinearAlgebra

function compute_reduced_density_matrix(psi::MPS, sites, i::Int, j::Int)
    """
    Compute the reduced density matrix for any two sites using SVD decomposition.
    This is the mathematically correct method that properly traces out all other sites.
    """
    
    # Ensure i < j
    if i > j
        i, j = j, i
    end
    
    N = length(sites)
    
    # Step 1: Move the orthogonality center to between sites i and j
    mid_point = div(i + j, 2)
    orthogonalize!(psi, mid_point)
    
    # Step 2: Group sites into three regions: [1...i-1], [i,j], [j+1...N]
    
    # Contract all tensors from 1 to i-1 (left environment)
    if i > 1
        left_env = psi[1]
        for k = 2:(i-1)
            left_env = left_env * psi[k]
        end
    else
        left_env = ITensor(1.0)  # Identity tensor if no left sites
    end
    
    # Contract tensors i and j (the subsystem of interest)
    if j == i + 1
        # Adjacent case
        center = psi[i] * psi[j]
    else
        # Non-adjacent: contract all sites from i to j
        center = psi[i]
        for k = (i+1):j
            center = center * psi[k]
        end
    end
    
    # Contract all tensors from j+1 to N (right environment) 
    if j < N
        right_env = psi[j+1]
        for k = (j+2):N
            right_env = right_env * psi[k]
        end
    else
        right_env = ITensor(1.0)  # Identity tensor if no right sites
    end
    
    # Step 3: Contract environments to trace out unwanted degrees of freedom
    # This gives us the effective wavefunction for the i,j subsystem
    
    # Contract left environment
    if i > 1
        center = left_env * center
    end
    
    # Contract right environment  
    if j < N
        center = center * right_env
    end
    
    # Step 4: Form the reduced density matrix
    # Extract the physical indices for sites i and j
    s_i = sites[i]
    s_j = sites[j]
    
    # Form ρ = |center⟩⟨center|
    rho_tensor = center * dag(prime(center, s_i, s_j))
    
    # Convert to matrix form
    C_rows = combiner(s_i, s_j)
    C_cols = combiner(prime(s_i), prime(s_j))
    
    rho_combined = (rho_tensor * C_rows) * dag(C_cols)
    rho_matrix = matrix(rho_combined)
    
    return Array(rho_matrix)
end

compute_reduced_density_matrix (generic function with 1 method)

In [4]:
test_rdm(compute_reduced_density_matrix)

Test Summary:           | Pass  Total   Time
Comprehensive RDM Tests |   24     24  43.0s


Test.DefaultTestSet("Comprehensive RDM Tests", Any[Test.DefaultTestSet("Product State", Any[], 5, false, false, true, 1.763327084969156e9, 1.76332708960842e9, false, "In[2]"), Test.DefaultTestSet("Bell State (Maximal Entanglement)", Any[], 4, false, false, true, 1.763327089608436e9, 1.763327091581333e9, false, "In[2]"), Test.DefaultTestSet("GHZ State (Tripartite Only)", Any[], 5, false, false, true, 1.763327091581346e9, 1.763327094080695e9, false, "In[2]"), Test.DefaultTestSet("Two Bell Pairs (Non-Adjacent)", Any[], 2, false, false, true, 1.763327094080715e9, 1.763327095413192e9, false, "In[2]"), Test.DefaultTestSet("Antiferromagnetic Chain", Any[], 3, false, false, true, 1.76332709541321e9, 1.763327126947413e9, false, "In[2]"), Test.DefaultTestSet("W State", Any[], 2, false, false, true, 1.763327126947431e9, 1.763327126950985e9, false, "In[2]"), Test.DefaultTestSet("Partial Trace Produces Mixed State", Any[], 1, false, false, true, 1.76332712695099e9, 1.763327126978393e9, false, "In[2

In [9]:
using ITensors, ITensorMPS
using LinearAlgebra

using ITensors

"""
    compute_reduced_density_matrix_efficient(
        psi::MPS,
        sites,
        i::Int,
        j::Int
    )

Calculates the reduced density matrix rho_ij for any two sites i and j (where i < j)
in an MPS 'psi'.

This is the efficient O(N*chi^3) method directly from ITensors documentation.
Converted from C++ code at: https://itensor.org/docs.cgi?page=formulas/mps_two_rdm
"""
function compute_reduced_density_matrix_efficient(
    psi::MPS,
    sites,
    i::Int,
    j::Int
)
    
end

# --- Example Usage (requires a full Julia environment) ---
#
# N = 10
# sites = siteinds("S=1/2", N)
# psi = randomMPS(sites, 10)
#
# # Calculate RDM for sites 3 and 7
# rho_37 = compute_reduced_density_matrix_efficient(psi, sites, 3, 7)
#
# println("Size of rho_37: ", size(rho_37)) # Should be 4x4
# println("Trace of rho_37: ", tr(rho_37)) # Should be 1.0
#

compute_reduced_density_matrix_efficient

In [22]:
test_rdm(compute_reduced_density_matrix_efficient)

GHZ State (Tripartite Only): Error During Test at In[2]:64
  Got exception outside of a @test
  DimensionMismatch: 
  Stacktrace:
   [1] matrix(T::ITensor)
     @ ITensors ~/.julia/packages/ITensors/Xs85I/src/tensor_operations/permutations.jl:182
   [2] compute_reduced_density_matrix_efficient(psi::MPS, sites::Vector{Index{Int64}}, i::Int64, j::Int64)
     @ Main ./In[9]:120
   [3] macro expansion
     @ ./In[2]:85 [inlined]
   [4] macro expansion
     @ ~/.julia/juliaup/julia-1.11.7+0.aarch64.apple.darwin14/share/julia/stdlib/v1.11/Test/src/Test.jl:1709 [inlined]
   [5] macro expansion
     @ ./In[2]:65 [inlined]
   [6] macro expansion
     @ ~/.julia/juliaup/julia-1.11.7+0.aarch64.apple.darwin14/share/julia/stdlib/v1.11/Test/src/Test.jl:1709 [inlined]
   [7] test_rdm(func::typeof(compute_reduced_density_matrix_efficient))
     @ Main ./In[2]:21
   [8] top-level scope
     @ In[22]:1
Two Bell Pairs (Non-Adjacent): Error During Test at In[2]:95
  Got exception outside of a @test
  Dime

GHZ State (Tripartite Only): Error During Test at In[2]:64
  Got exception outside of a @test
  DimensionMismatch: 
  Stacktrace:
   [1] matrix(T::ITensor)
     @ ITensors ~/.julia/packages/ITensors/Xs85I/src/tensor_operations/permutations.jl:182
   [2] compute_reduced_density_matrix_efficient(psi::MPS, sites::Vector{Index{Int64}}, i::Int64, j::Int64)
     @ Main ./In[9]:120
   [3] macro expansion
     @ ./In[2]:85 [inlined]
   [4] macro expansion
     @ ~/.julia/juliaup/julia-1.11.7+0.aarch64.apple.darwin14/share/julia/stdlib/v1.11/Test/src/Test.jl:1709 [inlined]
   [5] macro expansion
     @ ./In[2]:65 [inlined]
   [6] macro expansion
     @ ~/.julia/juliaup/julia-1.11.7+0.aarch64.apple.darwin14/share/julia/stdlib/v1.11/Test/src/Test.jl:1709 [inlined]
   [7] test_rdm(func::typeof(compute_reduced_density_matrix_efficient))
     @ Main ./In[2]:21
   [8] top-level scope
     @ In[22]:1
Two Bell Pairs (Non-Adjacent): Error During Test at In[2]:95
  Got exception outside of a @test
  Dime

LoadError: [91mSome tests did not pass: 12 passed, 0 failed, 6 errored, 0 broken.[39m

## Large System Test (N=50)

Let's verify that our RDM computation works efficiently for large systems like N=50, which would be impossible with full state vectors (2^50 ≈ 10^15 amplitudes!).

In [11]:
# Create an N=50 antiferromagnetic Heisenberg chain
N = 30
sites = siteinds("S=1/2", N; conserve_qns=true)
init_state = [isodd(i) ? "Up" : "Dn" for i in 1:N]
ψ = MPS(sites, init_state)

# Hamiltonian: J(SxSx + SySy + SzSz) with J=1
ampo = OpSum()
for i = 1:(N-1)
    ampo += 0.5, "S+", i, "S-", i+1
    ampo += 0.5, "S-", i, "S+", i+1
    ampo += "Sz", i, "Sz", i+1
end
H = MPO(ampo, sites)

# Find ground state with DMRG
sweeps = Sweeps(10)
setmaxdim!(sweeps, 20, 50, 100, 200)
setcutoff!(sweeps, 1E-10)
println("Running DMRG for N=$N chain...")
energy, ψ_gs = dmrg(H, ψ, sweeps; outputlevel=1)

println("\nGround state energy: $energy")
println("Max bond dimension: $(maxlinkdim(ψ_gs))")
println("\n✓ Successfully found ground state for N=50 system!")

Running DMRG for N=30 chain...
After sweep 1 energy=-13.008973451493146  maxlinkdim=4 maxerr=5.93E-16 time=0.031
After sweep 1 energy=-13.008973451493146  maxlinkdim=4 maxerr=5.93E-16 time=0.031
After sweep 2 energy=-13.10816129419098  maxlinkdim=16 maxerr=4.70E-11 time=0.065
After sweep 2 energy=-13.10816129419098  maxlinkdim=16 maxerr=4.70E-11 time=0.065
After sweep 3 energy=-13.11135478441103  maxlinkdim=47 maxerr=9.92E-11 time=0.089
After sweep 3 energy=-13.11135478441103  maxlinkdim=47 maxerr=9.92E-11 time=0.089
After sweep 4 energy=-13.111355751576554  maxlinkdim=47 maxerr=9.94E-11 time=0.101
After sweep 4 energy=-13.111355751576554  maxlinkdim=47 maxerr=9.94E-11 time=0.101
After sweep 5 energy=-13.11135575157317  maxlinkdim=47 maxerr=9.47E-11 time=0.107
After sweep 5 energy=-13.11135575157317  maxlinkdim=47 maxerr=9.47E-11 time=0.107
After sweep 6 energy=-13.11135575157277  maxlinkdim=47 maxerr=9.40E-11 time=0.151
After sweep 6 energy=-13.11135575157277  maxlinkdim=47 maxerr=9.4

In [12]:
ITensors.set_warn_order(51)

# Now compute reduced density matrices for various site pairs
println("\nComputing reduced density matrices:")

# Test 1: Adjacent sites (1, 2)
@time ρ_12 = compute_reduced_density_matrix_efficient(ψ_gs, sites, 1, 2)
C_12 = compute_concurrence(ρ_12)
println("Sites (1,2): trace=$(tr(ρ_12)), concurrence=$C_12")

# Test 2: Nearby sites (10, 15)
@time ρ_10_15 = compute_reduced_density_matrix_efficient(ψ_gs, sites, 10, 15)
C_10_15 = compute_concurrence(ρ_10_15)
println("Sites (10,15): trace=$(tr(ρ_10_15)), concurrence=$C_10_15")

# Test 3: Distant sites (1, 50) - opposite ends!
@time ρ_1_50 = compute_reduced_density_matrix_efficient(ψ_gs, sites, 1, 50)
C_1_50 = compute_concurrence(ρ_1_50)
println("Sites (1,50): trace=$(tr(ρ_1_50)), concurrence=$C_1_50")

# Test 4: Middle sites (25, 26)
@time ρ_25_26 = compute_reduced_density_matrix_efficient(ψ_gs, sites, 25, 26)
C_25_26 = compute_concurrence(ρ_25_26)
println("Sites (25,26): trace=$(tr(ρ_25_26)), concurrence=$C_25_26")

println("\n✓ All RDM computations completed successfully!")
println("✓ Each RDM is only 4×4, regardless of system size N=50")


Computing reduced density matrices:


LoadError: InterruptException: